In [ ]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas
import logging
import asyncio

In [51]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_user = os.getenv('SNOWFLAKE_USER')
account = os.getenv('SNOWFLAKE_ACCOUNT')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [52]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()


Data Engineer (26, 51)
Data Scientist (8, 45)
Data Analyst (28, 51)


In [53]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,seniority,directapply,linkedin_org_slug,salary_raw.@type,salary_raw.currency,salary_raw.value.@type,salary_raw.value.minValue,salary_raw.value.maxValue,salary_raw.value.unitText,job_category
0,1665658011,2025-05-16T11:07:25,2025-05-16T11:32:48.266835,Data Engineer-Enterprise Content Management - ...,IBM,https://www.linkedin.com/company/ibm,2025-06-15T11:07:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Not Applicable,False,ibm,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
1,1665657991,2025-05-16T11:07:10,2025-05-16T11:32:47.874046,Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Mid-Senior level,False,dws,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
2,1665658026,2025-05-16T11:07:10,2025-05-16T11:32:48.877412,Senior Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Mid-Senior level,False,dws,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
3,1665592008,2025-05-16T10:46:14,2025-05-16T11:03:13.80281,Senior Data Engineer,Fortescue,https://www.linkedin.com/company/fortescue,2025-06-15T10:46:14,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Mid-Senior level,False,fortescue,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
4,1664881973,2025-05-16T06:26:37,2025-05-16T06:33:00.663999,Data Engineer,Redwolf + Rosch,https://www.linkedin.com/company/redwolfroschr...,2025-06-15T06:26:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Mid-Senior level,True,redwolfroschrecruitment,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer


In [54]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['title'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(46, 51)

In [55]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [56]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(46, 51)

In [57]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1665658011,2025-05-16T11:07:25,2025-05-16T11:32:48.266835,Data Engineer-Enterprise Content Management - ...,Data Engineer,IBM,https://www.linkedin.com/company/ibm,2025-06-15T11:07:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Armonk, New York, NY",Public Company,NaN,"['Cloud', 'Mobile', 'Cognitive', 'Security', '...","['International Business Machines Corp., New O...","At IBM, we do more than work. We create. We cr...",False,Not Applicable,False,ibm
1,1665657991,2025-05-16T11:07:10,2025-05-16T11:32:47.874046,Data Engineer,Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Melbourne, Victoria",Public Company,1991.0,"['Cloud Solutions', 'Digital Transformation', ...","['550 Bourke St, Level 14, Melbourne, Victoria...","Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws


### Process the df_daily_all data 


In [58]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-16
1    2025-05-16
2    2025-05-16
3    2025-05-16
4    2025-05-16
Name: job_date, dtype: object

In [59]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1665658011,2025-05-16T11:07:25,2025-05-16T11:32:48.266835,Data Engineer-Enterprise Content Management - ...,Data Engineer,IBM,https://www.linkedin.com/company/ibm,2025-06-15T11:07:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Cloud', 'Mobile', 'Cognitive', 'Security', '...","['International Business Machines Corp., New O...","At IBM, we do more than work. We create. We cr...",False,Not Applicable,False,ibm,2025-05-16,Sydney,NSW
1,1665657991,2025-05-16T11:07:10,2025-05-16T11:32:47.874046,Data Engineer,Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Cloud Solutions', 'Digital Transformation', ...","['550 Bourke St, Level 14, Melbourne, Victoria...","Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws,2025-05-16,Sydney,NSW
2,1665658026,2025-05-16T11:07:10,2025-05-16T11:32:48.877412,Senior Data Engineer,Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Cloud Solutions', 'Digital Transformation', ...","['550 Bourke St, Level 14, Melbourne, Victoria...","Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws,2025-05-16,Sydney,NSW
3,1665592008,2025-05-16T10:46:14,2025-05-16T11:03:13.80281,Senior Data Engineer,Data Engineer,Fortescue,https://www.linkedin.com/company/fortescue,2025-06-15T10:46:14,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['Level 2, 87 Adelaide Terrace, East Perth, We...","We're a proud West Australian company, recogni...",False,Mid-Senior level,False,fortescue,2025-05-16,Perth,WA
4,1664881973,2025-05-16T06:26:37,2025-05-16T06:33:00.663999,Data Engineer,Data Engineer,Redwolf + Rosch,https://www.linkedin.com/company/redwolfroschr...,2025-06-15T06:26:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital Marketing', 'Permanent', 'Contract &...","['Level 2, 602 Bourke St, Melbourne, Victoria ...",Redwolf + Rosch (R+R) is a specialised Technol...,True,Mid-Senior level,True,redwolfroschrecruitment,2025-05-16,Sydney,NSW


In [60]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

41     FULL_TIME
42     FULL_TIME
43    CONTRACTOR
44     PART_TIME
45     FULL_TIME
Name: employment_type, dtype: object

In [61]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

41           11-50
42             nan
43    5,001-10,000
44       501-1,000
45       501-1,000
Name: org_size, dtype: object

In [62]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1665658011,2025-05-16T11:07:25,2025-05-16T11:32:48.266835,Data Engineer-Enterprise Content Management - ...,Data Engineer,IBM,https://www.linkedin.com/company/ibm,2025-06-15T11:07:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"At IBM, we do more than work. We create. We cr...",False,Not Applicable,False,ibm,2025-05-16,Sydney,NSW,FULL_TIME,"10,001+"
1,1665657991,2025-05-16T11:07:10,2025-05-16T11:32:47.874046,Data Engineer,Data Engineer,DWS,https://www.linkedin.com/company/dws,2025-06-15T11:07:10,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws,2025-05-16,Sydney,NSW,FULL_TIME,"501-1,000"


In [63]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1665658011,Data Engineer-Enterprise Content Management - ...,Data Engineer,2025-05-16,Sydney,NSW,FULL_TIME,IBM,https://www.linkedin.com/company/ibm,https://au.linkedin.com/jobs/view/data-enginee...,...,"Armonk, New York, NY",Public Company,NaN,"['Cloud', 'Mobile', 'Cognitive', 'Security', '...","['International Business Machines Corp., New O...","At IBM, we do more than work. We create. We cr...",False,Not Applicable,False,ibm
1,1665657991,Data Engineer,Data Engineer,2025-05-16,Sydney,NSW,FULL_TIME,DWS,https://www.linkedin.com/company/dws,https://au.linkedin.com/jobs/view/data-enginee...,...,"Melbourne, Victoria",Public Company,1991.0,"['Cloud Solutions', 'Digital Transformation', ...","['550 Bourke St, Level 14, Melbourne, Victoria...","Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws
2,1665658026,Senior Data Engineer,Data Engineer,2025-05-16,Sydney,NSW,FULL_TIME,DWS,https://www.linkedin.com/company/dws,https://au.linkedin.com/jobs/view/senior-data-...,...,"Melbourne, Victoria",Public Company,1991.0,"['Cloud Solutions', 'Digital Transformation', ...","['550 Bourke St, Level 14, Melbourne, Victoria...","Established in 1991, DWS is a leading Australi...",False,Mid-Senior level,False,dws
3,1665592008,Senior Data Engineer,Data Engineer,2025-05-16,Perth,WA,FULL_TIME,Fortescue,https://www.linkedin.com/company/fortescue,https://au.linkedin.com/jobs/view/senior-data-...,...,"East Perth, Western Australia",Public Company,2003.0,[''],"['Level 2, 87 Adelaide Terrace, East Perth, We...","We're a proud West Australian company, recogni...",False,Mid-Senior level,False,fortescue
4,1664881973,Data Engineer,Data Engineer,2025-05-16,Sydney,NSW,CONTRACTOR,Redwolf + Rosch,https://www.linkedin.com/company/redwolfroschr...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Melbourne, Victoria",Privately Held,2013.0,"['Digital Marketing', 'Permanent', 'Contract &...","['Level 2, 602 Bourke St, Melbourne, Victoria ...",Redwolf + Rosch (R+R) is a specialised Technol...,True,Mid-Senior level,True,redwolfroschrecruitment


## Connect to  Snowflake database for raw data initial processing

In [64]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user=snowflake_user,
            password=snowflake_password,
            account=account,
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [65]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_48993/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(622, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [ ]:
# df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [66]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [67]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

0 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG


In [ ]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

async def translate_text(translator, text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        result = await translator.translate(str(text), dest=target_language)
        return result.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

async def translate_column(df, col, target_language='en'):
    translator = Translator()
    unique_values = df[col].dropna().unique()

    tasks = {val: translate_text(translator, val, target_language) for val in unique_values}
    translation_map = {}
    for val, task in tasks.items():
        translation_map[val] = await task
    df[col] = df[col].map(translation_map).fillna('NA')

async def main():

    for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
        await translate_column(df_new_jobs, col, 'en')

await main()

In [69]:
df_new_jobs['CITY'].unique()

array([], dtype=float64)

In [70]:
df_new_jobs['STATE'].unique()

array([], dtype=float64)

In [71]:
df_new_jobs.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG


In [72]:
df_new_jobs.shape

(0, 31)

In [26]:
#SQLalchemy v2.0.40
# def load_to_snowflake(df_new_jobs):
#     # Create a Snowflake connection engine
#     engine = create_engine(
#         'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
#             user=snowflake_user,
#             password=snowflake_password,
#             account=account,
#             warehouse="SNOWFLAKE_LEARNING_WH",
#             database="linkedin_db",
#             schema="linkedin_raw"
#         )
#     )

#     table_name = "linkedin_job_api_cleaned_data"

#     df_new_jobs.to_sql(
#         name=table_name,
#         con=engine,
#         if_exists='append', #append data
#         index=False
#     )

#     print(f"Data loaded to Snowflake table {table_name} successfully.")


# load_to_snowflake(df_new_jobs)

In [ ]:
#Downgraded SQLalchemy to 1.4.45
def load_to_snowflake(df, table_name="LINKEDIN_JOB_API_CLEANED_DATA"):
    # Snowflake connection parameters
    conn_params = {
        'user': snowflake_user,
        'password': snowflake_password,
        'account': account,
        'warehouse': 'SNOWFLAKE_LEARNING_WH',
        'database': 'linkedin_db',
        'schema': 'linkedin_raw'
    }

    # Create Snowflake connection
    conn = connect(**conn_params)

    # Ensure column names are uppercase to match Snowflake
    df.columns = df.columns.str.upper()

    # Check if the table exists
    try:
        with conn.cursor() as cursor:
            cursor.execute(f"DESC TABLE {table_name}")
            logging.info("Table %s exists.", table_name)
    except Exception as e:
        logging.error("Table %s does not exist or cannot be accessed: %s", table_name, e)
        conn.close()
        return

    # Load DataFrame to Snowflake
    try:
        # Append the DataFrame to the existing table
        write_pandas(conn, df, table_name, auto_create_table=False)
        logging.info("Successfully loaded %d rows to Snowflake", len(df))
    except Exception as e:
        logging.error("An error occurred while loading data to Snowflake: %s", e)
    finally:
        conn.close()  # Ensure the connection is closed after loading

# Example usage
load_to_snowflake(df_new_jobs)